# 类似于youtube Candidate Generation Model 附上网址 https://zhuanlan.zhihu.com/p/52169807
## data： 链接：https://pan.baidu.com/s/1e3onk82FT3IhSrcL-6H_OQ  提取码：ye70 

In [ ]:
from __future__ import print_function
import sys
import linecache
import numpy as np
import tensorflow as tf
import time
import math

In [2]:
train_file = "data/dbpedia.train"
test_file = "data/dbpedia.test"
label_dict = {}
sku_dict = {}

max_window_size = 1000
batch_size = 500
emb_size = 128

# Parameters
learning_rate = 0.01
training_epochs = 1
display_step = 1

# Network Parameters
n_hidden_1 = 128 # 1st layer number of features
# n_hidden_2 = 256 # 2nd layer number of features

In [3]:
def init_data(read_file):
    #0 is used for padding embedding
    label_cnt = 0
    sku_cnt = 1
    f = open(read_file,'r')
    for line in f:
        line = line.strip().split(' ')
        for i in line:
            if i.find('__label__') == 0:
                if i not in label_dict:
                    label_dict[i] = label_cnt
                    label_cnt += 1
            else:
                if i not in sku_dict:
                    sku_dict[i] = sku_cnt
                    sku_cnt += 1

def read_data(pos, batch_size, data_lst):
    batch = data_lst[pos:pos + batch_size]
    x = np.zeros((batch_size, max_window_size))
    mask = np.zeros((batch_size, max_window_size))
    y = []
    word_num = np.zeros((batch_size))
    line_no = 0
    for line in batch:
        line = line.strip().split(' ')
        y.append(label_dict[line[0]])
        col_no = 0
        for i in line[1:]:
            if i in sku_dict:
                x[line_no][col_no] = sku_dict[i]
                mask[line_no][col_no] = 1
                col_no += 1
            if col_no >= max_window_size:
                break
        word_num[line_no] = col_no
        line_no += 1

    return x, np.array(y).reshape(batch_size, 1), mask.reshape(batch_size, max_window_size, 1), word_num.reshape(batch_size, 1)


In [4]:
#========================
init_data(train_file)
n_classes = len(label_dict)
train_lst = linecache.getlines(train_file)
print("Class Num: ", n_classes)

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([emb_size, n_hidden_1])),
    # 'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    # 'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


Class Num:  14


In [5]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    #x = tf.nn.dropout(x, 0.8)
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    #dlayer_1 = tf.nn.dropout(layer_1, 0.5)
    #layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    # out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    # return out_layer
    return layer_1

embedding = {
    'input':tf.Variable(tf.random_uniform([len(sku_dict)+1, emb_size], -1.0, 1.0))
    # 'output':tf.Variable(tf.random_uniform([len(label_dict)+1, emb_size], -1.0, 1.0))
}


In [6]:
emb_mask = tf.placeholder(tf.float32, shape=[None, max_window_size, 1])
word_num = tf.placeholder(tf.float32, shape=[None, 1])

x_batch = tf.placeholder(tf.int32, shape=[None, max_window_size])
y_batch = tf.placeholder(tf.int64, [None, 1])

input_embedding = tf.nn.embedding_lookup(embedding['input'], x_batch)
project_embedding = tf.div(tf.reduce_sum(tf.multiply(input_embedding,emb_mask), 1),word_num)# batch*max_window_size*emb_size ==>bath*emb_size

# Construct model
pred = multilayer_perceptron(project_embedding, weights, biases)#bath*emb_size  (dot) emb_size*n_hidden_1 +biases ==>bath*n_hidden_1

# Construct the variables for the NCE loss
nce_weights = tf.Variable(
    tf.truncated_normal([n_classes, n_hidden_1],
                        stddev=1.0 / math.sqrt(n_hidden_1)))
nce_biases = tf.Variable(tf.zeros([n_classes]))

loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=y_batch,
                     inputs=pred,
                     num_sampled=10,
                     num_classes=n_classes))

cost = tf.reduce_sum(loss) / batch_size
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
out_layer = tf.matmul(pred, tf.transpose(nce_weights)) + nce_biases


W0811 14:44:27.652238 140377874786112 deprecation.py:323] From <ipython-input-6-77618e93b8c9>:8: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0811 14:44:27.738372 140377874786112 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
init = tf.global_variables_initializer()
#with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    start_time = time.time()
    total_batch = int(len(train_lst) / batch_size)
    print("total_batch of training data: ", total_batch)
    for epoch in range(training_epochs):
        avg_cost = 0.
        for i in range(total_batch):
            x, y, batch_mask, word_number = read_data(i * batch_size, batch_size, train_lst)
            _,c = sess.run([optimizer, cost], feed_dict={x_batch: x, emb_mask: batch_mask, word_num: word_number, y_batch: y})
            #print("Epoch %d Batch %d Elapsed time %fs" %(epoch, i, time.time() - start_time))
            # Compute average loss
            avg_cost += c / total_batch
            # correct_prediction = tf.equal(tf.argmax(out_layer, 1), tf.reshape(y_batch, [batch_size]))
            # accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            # print("Accuracy:", accuracy.eval({x_batch: x, y_batch: y, emb_mask: batch_mask, word_num: word_number}))

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", \
              "{:.9f}".format(avg_cost))

    # Test model
    correct_prediction = tf.equal(tf.argmax(out_layer, 1), tf.reshape(y_batch, [batch_size]))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    test_lst = linecache.getlines(test_file)
    total_batch = int(len(test_lst) / batch_size)
    final_accuracy = 0
    for i in range(total_batch):
        x, y, batch_mask, word_number = read_data(i*batch_size, batch_size, test_lst)
        batch_accuracy = accuracy.eval({x_batch: x, y_batch: y, emb_mask: batch_mask, word_num: word_number})
        print("Batch Accuracy: ", batch_accuracy)
        final_accuracy += batch_accuracy
    print("Final Accuracy: ", final_accuracy * 1.0 / total_batch)

total_batch of training data:  1120
Epoch: 0001 cost= 0.002873009
Batch Accuracy:  0.982
Batch Accuracy:  0.976
Batch Accuracy:  0.988
Batch Accuracy:  0.982
Batch Accuracy:  0.98
Batch Accuracy:  0.982
Batch Accuracy:  0.972
Batch Accuracy:  0.98
Batch Accuracy:  0.974
Batch Accuracy:  0.988
Batch Accuracy:  0.982
Batch Accuracy:  0.982
Batch Accuracy:  0.966
Batch Accuracy:  0.988
Batch Accuracy:  0.986
Batch Accuracy:  0.988
Batch Accuracy:  0.986
Batch Accuracy:  0.984
Batch Accuracy:  0.982
Batch Accuracy:  0.976
Batch Accuracy:  0.986
Batch Accuracy:  0.984
Batch Accuracy:  0.978
Batch Accuracy:  0.98
Batch Accuracy:  0.982
Batch Accuracy:  0.986
Batch Accuracy:  0.984
Batch Accuracy:  0.976
Batch Accuracy:  0.966
Batch Accuracy:  0.98
Batch Accuracy:  0.99
Batch Accuracy:  0.978
Batch Accuracy:  0.984
Batch Accuracy:  0.986
Batch Accuracy:  0.974
Batch Accuracy:  0.98
Batch Accuracy:  0.984
Batch Accuracy:  0.988
Batch Accuracy:  0.986
Batch Accuracy:  0.974
Batch Accuracy:  0.9

In [19]:
x.shape

(500, 1000)

In [62]:
n_classes

14

## 小案例

In [37]:
embedding2 = {
    'input':tf.Variable(tf.random_uniform([len(sku_dict)+1, emb_size], -1.0, 1.0))
    # 'output':tf.Variable(tf.random_uniform([len(label_dict)+1, emb_size], -1.0, 1.0))
}

In [42]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(embedding2['input'][0,:])

array([ 0.8758478 ,  0.34019566,  0.082232  ,  0.68539095, -0.3927145 ,
        0.7513776 , -0.60025716, -0.47553897, -0.10748911,  0.01165295,
        0.8743646 , -0.6310854 , -0.85378003,  0.7571566 , -0.63642955,
        0.8241277 ,  0.45026827,  0.25037718, -0.33855772,  0.2109375 ,
        0.7913141 ,  0.11828518,  0.43765187,  0.81121707, -0.2500875 ,
       -0.05734873, -0.1650126 ,  0.38498878,  0.37742782,  0.9495721 ,
       -0.41694784, -0.77679896,  0.8964803 , -0.00870299, -0.1644988 ,
        0.12021565,  0.5084343 ,  0.08331561,  0.01648998, -0.7906444 ,
        0.18836236,  0.60497046, -0.18774438,  0.5860958 ,  0.72163534,
        0.09724617, -0.65918446,  0.7554071 ,  0.58323264,  0.6362982 ,
        0.32701492, -0.61844397, -0.20653105, -0.8768456 , -0.90562415,
        0.49605894, -0.44254804, -0.99577117,  0.9388304 ,  0.1828866 ,
       -0.2825377 , -0.6500766 ,  0.3259263 , -0.08249831, -0.1181612 ,
       -0.72467494, -0.5125823 ,  0.36024046, -0.84710145,  0.34

In [51]:

c = np.random.random([5,5])
b = tf.nn.embedding_lookup(c, [[1,1,1,0],[2,1,0,0],[3,2,1,0]])

with tf.Session() as sess:
	sess.run(tf.initialize_all_variables())
	print(sess.run(b))
    
	print(c)

[[[0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
  [0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
  [0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
  [0.14551133 0.38358059 0.52439619 0.92029868 0.86354287]]

 [[0.26615393 0.17822771 0.5855087  0.29269982 0.02404527]
  [0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
  [0.14551133 0.38358059 0.52439619 0.92029868 0.86354287]
  [0.14551133 0.38358059 0.52439619 0.92029868 0.86354287]]

 [[0.24434086 0.1200454  0.65374472 0.39212664 0.69070037]
  [0.26615393 0.17822771 0.5855087  0.29269982 0.02404527]
  [0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
  [0.14551133 0.38358059 0.52439619 0.92029868 0.86354287]]]
[[0.14551133 0.38358059 0.52439619 0.92029868 0.86354287]
 [0.82630656 0.6855396  0.05022895 0.21493632 0.01809424]
 [0.26615393 0.17822771 0.5855087  0.29269982 0.02404527]
 [0.24434086 0.1200454  0.65374472 0.39212664 0.69070037]
 [0.62156907 0.55158928 0.50647869 0.2959347  0.465715

In [54]:
cc = np.random.random([5,4,2])
cc

array([[[0.55544131, 0.8304293 ],
        [0.68217629, 0.95945437],
        [0.2687992 , 0.8579725 ],
        [0.84239537, 0.66280121]],

       [[0.80892266, 0.99986152],
        [0.07148699, 0.85757265],
        [0.86742756, 0.20367565],
        [0.04267489, 0.83615199]],

       [[0.45284969, 0.7625072 ],
        [0.8613855 , 0.1709657 ],
        [0.90207779, 0.2061405 ],
        [0.5995104 , 0.8500277 ]],

       [[0.57260413, 0.31813009],
        [0.52549597, 0.71890025],
        [0.08609023, 0.7512856 ],
        [0.74849522, 0.62619595]],

       [[0.22367012, 0.51006928],
        [0.75731616, 0.38349344],
        [0.25053126, 0.69025291],
        [0.78617543, 0.989679  ]]])

In [64]:
hh=tf.reduce_sum(cc,1)
nce_biases1 = tf.Variable(tf.zeros([n_classes,2]))
with tf.Session() as sess:
	sess.run(tf.initialize_all_variables())
	print(sess.run(hh))
	print(sess.run(nce_biases1))    

[[2.34881217 3.31065738]
 [1.79051211 2.89726181]
 [2.81582338 1.98964109]
 [1.93268556 2.41451189]
 [2.01769298 2.57349463]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [58]:
hh=tf.reduce_sum(cc,0)
with tf.Session() as sess:
	sess.run(tf.initialize_all_variables())
	print(sess.run(hh))

[[2.61348792 3.42099739]
 [2.8978609  3.0903864 ]
 [2.37492604 2.70932716]
 [3.01925132 3.96485585]]


In [60]:
c = tf.truncated_normal(shape=[2,2,1,5], mean=0, stddev=1)
 
with tf.Session() as sess:
	print( sess.run(c))

[[[[-1.9546583   0.33155093  0.95034134 -0.08760493  1.6392941 ]]

  [[ 0.38250458 -0.07407135 -0.63093686  0.08415447 -1.6335772 ]]]


 [[[ 1.962625    1.2704955  -1.1871498  -0.41778094  0.7825305 ]]

  [[-0.5111942   0.602534   -0.09397011  1.4350804   1.3380942 ]]]]
